#### Loading Data

In [13]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggpubr))
suppressPackageStartupMessages(library(dplyr))
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/trMeanCorUPGMANorm.RData")

#### Reading all the nwk files for the trees

In [14]:
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
save_dir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees"
treesCons0 <- read.tree(file.path(dir, "no_threshold0", "cluster_nwk.txt"))
txpsCons0 <- rownames(seBrainSim)[as.numeric(unlist(lapply(treesCons0, function(tr) tr$tip)))+1]

treesCons <- read.tree(file.path(dir, "no_threshold", "cluster_nwk.txt"))
txpsCons <- rownames(seBrainSim)[as.numeric(unlist(lapply(treesCons, function(tr) tr$tip)))+1]

treesMeanInf0 <- read.tree(file.path(dir, "no_threshold_meaninf_0", "group_nwk.txt"))
txpsMeanInf0 <- rownames(seBrainSim)[as.numeric(unlist(lapply(treesMeanInf0, function(tr) tr$tip)))+1]

treesThr <- read.tree(file.path(dir, "threshold", "cluster_nwk.txt"))
txpsThr <- rownames(seBrainSim)[as.numeric(unlist(lapply(treesThr, function(tr) tr$tip)))+1]
                                                       
groupsThr <- read.delim(file.path(dir, "old", "1_1", "clusters.txt"), header = F)
txpsOld <- unlist(sapply(strsplit(groupsThr$V1, split=",", fixed=T), function(x) x[2:length(x)]))
# txpsOld <- gsub("\\.[0-9]+", "", txpsOld)
txpsCor <- trMeanCorUPGMANorm$tip

In [4]:
gse <- suppressMessages(summarizeToGene(seBrainSim))
gse <- fishpond::computeInfRV(gse)
save(gse, file = file.path(save_dir, "gene_summ.RData"))

In [15]:
se <- labelKeep(seBrainSim)

In [46]:
length(txpsThr)
length(txpsOld)
length(txpsMeanInf0)
length(txpsCons0)
length(txpsCons)
length(txpsCor)
length(rownames(se)[mcols(se)[["keep"]]])

[1] 27156

[1] 13612

[1] 135138

[1] 134377

[1] 121784

[1] 73205

[1] 61893

In [47]:
txpsAll <- Reduce(union,list(txpsCons0, txpsCons, txpsThr, txpsMeanInf0, txpsOld, txpsCor, rownames(se)[mcols(se)[["keep"]]]))
length(txpsAll)
seBrainSimScaled <- scaleInfReps(seBrainSim)

[1] 139535

In [6]:
treeCons <- mergeTree(treesCons, se = mapDf)
mb <- mergeLeaves(treeCons, seBrainSim[txpsAll,])
treeCons <- mb[["tree"]]
yConsNS <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yConsNS) != treeCons$tip)
yAggConsNS <- prepSwish(treeCons, seBrainSim[treeCons$tip,])
yAggConsNS <- fishpond::computeInfRV(yAggConsNS, meanVariance=F)
yAggCons <- prepSwish(treeCons, seBrainSimScaled[treeCons$tip,])
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggConsNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggCons)[["condition"]])
rm(yAggCons)
gc()
save(yAggConsNS, file=file.path(save_dir, "yAggConsNS.RData"))
save(treeCons, file=file.path(save_dir, "treeCons.RData"))

[1] "Missing txps 17751"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,10970110,585.9,17712615,946.0,17712615,946
Vcells,715928664,5462.2,1177331421,8982.4,913559978,6970


[1] 0

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11037959,589.5,25029222,1336.8,25029222,1336.8
Vcells,1318892864,10062.4,2441743874,18629.1,2440748719,18621.5


#### Consensus Tree 0

In [7]:
treeCons0 <- mergeTree(treesCons0, se = mapDf)
mb <- mergeLeaves(treeCons0, seBrainSim[txpsAll,])
treeCons0 <- mb[["tree"]]
yConsNS0 <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yConsNS0) != treeCons0$tip)
yAggConsNS0 <- prepSwish(treeCons0, seBrainSim[treeCons0$tip,])
yAggConsNS0 <- fishpond::computeInfRV(yAggConsNS0, meanVariance=F)
yAggCons0 <- prepSwish(treeCons0, seBrainSimScaled[treeCons0$tip,])

infReps <- assays(yAggCons0)[grep("infRep", assayNames(yAggCons0))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggConsNS0)[["LFC"]] <- getLog2FC(infReps, colData(yAggCons0)[["condition"]])
rm(yAggCons0)
gc()

save(yAggConsNS0, file=file.path(save_dir, "yAggConsNS0.RData"))
save(treeCons0, file=file.path(save_dir, "treeCons0.RData"))

[1] "Missing txps 5158"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11039412,589.6,25029222,1336.8,25029222,1336.8
Vcells,1506975097,11497.4,2441743874,18629.1,2440748719,18621.5


[1] 0

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11061037,590.8,25029222,1336.8,25029222,1336.8
Vcells,1881028751,14351.2,4219624612,32193.2,3919046791,29900.0


#### Mean Inf Tree 0

In [8]:
treeMeanInf0 <- mergeTree(treesMeanInf0, se = mapDf)
mb <- mergeLeaves(treeMeanInf0, seBrainSim[txpsAll,])
treeMeanInf0 <- mb[["tree"]]
yMeanInfNS0 <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yMeanInfNS0) != treeMeanInf0$tip)

yAggMeanInfNS0 <- prepSwish(treeMeanInf0, seBrainSim[treeMeanInf0$tip,])
yAggMeanInfNS0 <- fishpond::computeInfRV(yAggMeanInfNS0, meanVariance=F)

yAggMeanInf0 <- prepSwish(treeMeanInf0, seBrainSimScaled[treeMeanInf0$tip,])

infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggMeanInfNS0)[["LFC"]] <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])

rm(yAggMeanInf0)
gc()

save(yAggMeanInfNS0, file=file.path(save_dir, "yAggMeanInfNS0.RData"))
save(treeMeanInf0, file=file.path(save_dir, "treeMeanInf0.RData"))

[1] "Missing txps 4397"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11061817,590.8,25029222,1336.8,25029222,1336.8
Vcells,2068857757,15784.2,4219624612,32193.2,3919046791,29900.0


[1] 0

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11066449,591.1,25029222,1336.8,25029222,1336.8
Vcells,2427611627,18521.3,5063629534,38632.5,4534170726,34593.0


#### Correlation

In [9]:
mb <- mergeLeaves(trMeanCorUPGMANorm, seBrainSim[txpsAll,])
treeCor <- mb[["tree"]]
rm(mb)
gc()

yAggCorNS <- prepSwish(treeCor, seBrainSim[treeCor$tip,])
yAggCorNS <- fishpond::computeInfRV(yAggCorNS, meanVariance=F)

yAggCor <- prepSwish(treeCor, seBrainSimScaled[treeCor$tip,])

infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)

mcols(yAggCorNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
rm(yAggCor)
gc()

save(yAggCorNS, file=file.path(save_dir, "yAggCorNS.RData"))
save(treeCor, file=file.path(save_dir, "treeCor.RData"))

[1] "Missing txps 66330"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11066343,591.1,25029222,1336.8,25029222,1336.8
Vcells,2428176105,18525.6,5063629534,38632.5,4534170726,34593.0


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11067186,591.1,25029222,1336.8,25029222,1336.8
Vcells,2662405906,20312.6,5063629534,38632.5,4534170726,34593.0


#### Threshold

In [10]:
treeThr <- mergeTree(treesThr, se = mapDf)
mb <- mergeLeaves(treeThr, seBrainSim[txpsAll,])
treeThr<- mb[["tree"]]
yThrNS <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yThrNS) != treeThr$tip)

yAggThrNS <- prepSwish(treeThr, seBrainSim[treeThr$tip,])
yAggThrNS <- fishpond::computeInfRV(yAggThrNS, meanVariance=F)

yAggThr <- prepSwish(treeThr, seBrainSimScaled[treeThr$tip,])

infReps <- assays(yAggThr)[grep("infRep", assayNames(yAggThr))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggThrNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggThrNS)[["condition"]])

rm(yAggThr)
gc()

save(yAggThrNS, file=file.path(save_dir, "yAggThrNS.RData"))
save(treeThr, file=file.path(save_dir, "treeThr.RData"))

[1] "Missing txps 112379"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11068016,591.1,25029222,1336.8,25029222,1336.8
Vcells,2850301068,21746.1,5063629534,38632.5,4534170726,34593.0


[1] 0

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11068886,591.2,25029222,1336.8,25029222,1336.8
Vcells,2997542628,22869.5,5063629534,38632.5,4996044166,38116.8


#### Terminus

In [94]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
mtxps <- setdiff(txpsAll, txpsOld)

yTermThrNS <- seBrainSim[txpsAll,]
groupsClust <- parseClustFile(file.path(dir, "old", "1_1", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groups))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)
rownames(yAggTermThrNS)[length(txpsAll)+1:length(groupsClust)] <- as.character(length(txpsAll)+1:length(groupsClust))
yAggTermThrNS <- fishpond::computeInfRV(yAggTermThrNS)
save(yAggTermThrNS, file = file.path(save_dir, "yAggTermThrNS.RData") )

In [95]:
yAggTermThrNS

class: SummarizedExperiment 
dim: 144935 12 
metadata(7): tximetaInfo quantInfo ... txdbInfo infRepsScaled
assays(105): counts abundance ... mean variance
rownames(144935): ENST00000591598.5 ENST00000075430.11 ... 144934
  144935
rowData names(1): meanInfRV
colnames(12): 1_1 2_1 ... 5_2 6_2
colData names(2): names condition

In [79]:
### NGenes
nNodes <- length(treeCons$tip)+1:treeCons$Nnode
desc <- Descendants(treeCons, nNodes)
genesCons <- mclapply(desc, function(txps) unique(mapDf[treeCons$tip.label[txps],"GENEID"]), mc.cores=1)
save(genesCons, file=file.path(save_dir,"genesCons.RData"))

In [32]:
### NGenes
nNodes <- length(treeCons0$tip)+1:treeCons0$Nnode
desc <- Descendants(treeCons0, nNodes)
genesCons0 <- mclapply(desc, function(txps) unique(mapDf[treeCons0$tip.label[txps],"GENEID"]), mc.cores=1)
save(genesCons0, file=file.path(save_dir,"genesCons0.RData"))

In [33]:
### NGenes
nNodes <- length(treeMeanInf0$tip)+1:treeMeanInf0$Nnode
desc <- Descendants(treeMeanInf0, nNodes)
genesMeanInf0 <- mclapply(desc, function(txps) unique(mapDf[treeMeanInf0$tip.label[txps],"GENEID"]), mc.cores=1)
save(genesMeanInf0, file=file.path(save_dir,"genesMeanInf0.RData"))

In [34]:
### NGenes
nNodes <- length(treeCor$tip)+1:treeCor$Nnode
desc <- Descendants(treeCor, nNodes)
genesCor <- mclapply(desc, function(txps) unique(mapDf[treeCor$tip.label[txps],"GENEID"]), mc.cores=1)
save(genesCor, file=file.path(save_dir,"genesCor.RData"))

In [35]:
### NGenes
nNodes <- length(treeThr$tip)+1:treeThr$Nnode
desc <- Descendants(treeThr, nNodes)
genesThr <- mclapply(desc, function(txps) unique(mapDf[treeThr$tip.label[txps],"GENEID"]), mc.cores=1)
save(genesThr, file=file.path(save_dir,"genesThr.RData"))